Лабораторна робота №4

Роботу виконали студенти групи ФБ - 35: Церман Марія, Кохта Андрій, Ворона Сергій.

In [59]:
import random

Генерація простих чисел та розбиття на пари

In [60]:
def power(a, b, m):
    res = 1
    a %= m
    while b > 0:
        if b % 2 == 1:
            res = (res * a) % m
        a = (a * a) % m
        b //= 2

    return res

def miller_rabin_test(n, k=40):
    if n == 2 or n == 3:
        return True
    if n % 2 == 0 or n < 2:
        return False

    s = 0
    d = n - 1
    while d % 2 == 0:
        d //= 2
        s += 1

    for _ in range(k):
        a = random.randrange(2, n - 1)
        x = power(a, d, n)

        if x == 1 or x == n - 1:
            continue

        for r in range(s - 1):
            x = power(x, 2, n)
            if x == n - 1:
                break
        else:
            return False

    return True


def get_random_prime(bits):
    while True:
        candidate = random.getrandbits(bits)
        candidate |= (1 << (bits - 1)) | 1

        small_primes = [3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53]
        is_composite = False
        for p in small_primes:
            if candidate % p == 0 and candidate != p:
                is_composite = True
                break

        if is_composite:
            continue

        if miller_rabin_test(candidate):
            return candidate

In [61]:
p, q, p1, q1 = get_random_prime(256), get_random_prime(256), get_random_prime(256), get_random_prime(256)

In [62]:
def check_keys(p, q, p1, q1):
    n = p * q
    n1 = p1 * q1

    if n > n1:
        return (p1, q1), (p, q), n1, n
    else:
        return (p, q), (p1, q1), n, n1

In [63]:
pair_A, pair_B, mult_A, mult_B = check_keys(p, q, p1, q1)
print("Пара для абонента А:")
print(f"p: {pair_A[0]}\n"
      f"q: {pair_A[1]}\n"
      f"pq: {mult_A}\n")

print("Пара для абонента B:")
print(f"p1: {pair_B[0]}\n"
      f"q1: {pair_B[1]}\n"
      f"pq1: {mult_B}\n")

print("pq < pq1:", ["Ні", "Так"][mult_A <= mult_B])

Пара для абонента А:
p: 96078532015062351918540592330886259298853782081909764520721624151783343804007
q: 76941414158374052694855588955098806720798436388727268447380994801313986804629
pq: 7392418123499513146975980645592658875639573574789561802471225518331943407882927188809679010874892150126240039660378231413403948559492794580532083576348403

Пара для абонента B:
p1: 89035336823785183585410792038048893814923078725170087512803440632812268506221
q1: 100337230616510337979712878471295932866676066932009624116727982474266537850601
pq1: 8933559123906809033370966558344309483453674012609457799523448164981173307290627110312430019898726527933942786938119854773687517564414337292345030337088821

pq < pq1: Так


Генерація пар ключів(публічного та секретного) для RSA

In [64]:
def euclid_extended(a, n):
    r0, r1 = a, n
    u0, v0 = 1, 0
    u1, v1 = 0, 1

    while r1 != 0:
        q = r0 // r1
        (r0, r1) = (r1, r0 - q * r1)

        (u0, u1) = (u1, u0 - q * u1)
        (v0, v1) = (v1, v0 - q * v1)

    gcd = r0
    u = u0
    v = v0

    return gcd, u, v

def generate_key_pair(p, q):
    n = p * q
    phi = (p - 1) * (q - 1)
    e = 65537
    gcd, u, _ = euclid_extended(e, phi)

    while gcd != 1:
        e = random.randrange(3, phi, 2)
        gcd, u, _ = euclid_extended(e, phi)

    d = u % phi

    return (e, n), (d, p, q)

In [65]:
public_A, private_A = generate_key_pair(pair_A[0], pair_A[1])
public_B, private_B = generate_key_pair(pair_B[0], pair_B[1])

for pub, priv, name in [(public_A, private_A, "A"), (public_B, private_B, "B")]:
    e, n = pub
    d, p, q = priv

    print(f"Абонент {name}:")
    print("Відкритий ключ")
    print(f"e: {e}\n"
          f"n: {n}\n")

    print("Приватний ключ")
    print(f"d: {d}\n"
          f"p: {p}\n"
          f"q: {q}\n")

Абонент A:
Відкритий ключ
e: 65537
n: 7392418123499513146975980645592658875639573574789561802471225518331943407882927188809679010874892150126240039660378231413403948559492794580532083576348403

Приватний ключ
d: 3291209179661394244510202836216222907264773757804138643399994173884819945301173400355514681048778453127330659092637664582598595659260226468330128945148465
p: 96078532015062351918540592330886259298853782081909764520721624151783343804007
q: 76941414158374052694855588955098806720798436388727268447380994801313986804629

Абонент B:
Відкритий ключ
e: 65537
n: 8933559123906809033370966558344309483453674012609457799523448164981173307290627110312430019898726527933942786938119854773687517564414337292345030337088821

Приватний ключ
d: 6506774970178183658069940914231766313428714700366203192127382619088626678825728290485059424499434250440373101084080652061406535957776979772681147723697473
p: 89035336823785183585410792038048893814923078725170087512803440632812268506221
q: 1003372306165103379

Шифрування, розшифрування, цифровий підпис, перевірка

In [66]:
def Encrypt(message, public_key):
    e, n = public_key
    if message >= n:
        raise ValueError("Помилка: Повідомлення M має бути меншим за модуль n")

    return power(message, e, n)

def Decrypt(ciphertext, private_key):
    if len(private_key) == 3:
        d, p, q = private_key
        n = p * q
    else:
        d, n = private_key

    return power(ciphertext, d, n)

def Sign(message, private_key):
    if len(private_key) == 3:
        d, p, q = private_key
        n = p * q
    else:
        d, n = private_key

    return power(message, d, n)

def Verify(message, signature, public_key):
    e, n = public_key
    check_val = power(signature, e, n)

    return check_val == message

In [67]:
subscribers = [("А", public_A, private_A), ("В", public_B, private_B)]

for name, pub_key, priv_key in subscribers:
    # Генерація повідомлення
    n_module = pub_key[1]
    message = random.randint(100, n_module // 2)
    print(f"Відкрите повідомлення {name}: {message}")

    # Шифрування
    crypto = Encrypt(message, pub_key)
    print(f"Криптограма {name}: {crypto}")

    # Розшифрування
    decrypted = Decrypt(crypto, priv_key)
    print(f"Розшифроване повідомлення {name}: {decrypted}")
    print(["Не співпадають\n", "Співпадають\n"][decrypted == message])

    # Цифровий підпис
    signature = Sign(message, priv_key)
    print(f"Цифровий підпис повідомлення {name}: {signature}")

    # Перевірка підпису
    is_valid = Verify(message, signature, pub_key)
    status = (["!Невалідний!", "!Валідний!"][is_valid])
    print(f"Підпис {name}: {status}\n")

Відкрите повідомлення А: 2256810425627285135368717359612272298110799919179755984452060340452338977973703680469935012308728752498654513375737453195696784771164853559121951016250079
Криптограма А: 4313987842243184151415884704669153105029739798526914838713240062379589492104909437080617139520524834658731042412196559691982541255206776280971086754059544
Розшифроване повідомлення А: 2256810425627285135368717359612272298110799919179755984452060340452338977973703680469935012308728752498654513375737453195696784771164853559121951016250079
Співпадають

Цифровий підпис повідомлення А: 6233284276991947741644704460277974133185131201024102557839512619202189009035456742581376518146752063887189202241109826288375299388145604545348289850738367
Підпис А: !Валідний!

Відкрите повідомлення В: 3334141605232885286237567127521331880347196189874299784731536913467469472590613073297933551746486948420595210124611623613483066562774571974222332912317269
Криптограма В: 7516894616252755814209544482056174783042225252351

Конфіденційне розсилання ключів

In [68]:
def SendKey(k, private_key_A, public_key_B):
    S = Sign(k, private_key_A)
    k1 = Encrypt(k, public_key_B)
    S1 = Encrypt(S, public_key_B)

    return k1, S1

def ReceiveKey(package, private_key_B, public_key_A):
    k1, S1 = package
    k = Decrypt(k1, private_key_B)
    S = Decrypt(S1, private_key_B)
    is_valid = Verify(k, S, public_key_A)

    if is_valid:
        return k, "Підпис вірний, ключ отримано!"
    else:
        return None, "Підпис невірний!"

In [69]:
# 0<k<n
n_a = public_A[1]
secret_k = random.randint(100, n_a // 2)
print(f"Секретний ключ k: {secret_k}\n")

print("Відправляє А:")
package = SendKey(secret_k, private_A, public_B)
print(f"Шифрований ключ: {package[0]}\n"
      f"Цифровий підпис: {package[1]}\n")

print("Отримує B:")
received_k, status_msg = ReceiveKey(package, private_B, public_A)
print(f"Відкритий ключ k: {received_k}")
print(status_msg)

Секретний ключ k: 1829364036164169935248855471102456317560319984073539796048026974387863837794884786469725285704259974630240481748162306413246647833386659341461395148758063

Відправляє А:
Шифрований ключ: 7014645197503561938497578208622627402742387856384216637543462136960810800630631968804395019559015960659555987378889498380254306855483002285610280614920182
Цифровий підпис: 2251864940557865070022660021176113993692483335092449365597803620888866245407321647954025772912928874567511016956949520230442687124487824565064189413970134

Отримує B:
Відкритий ключ k: 1829364036164169935248855471102456317560319984073539796048026974387863837794884786469725285704259974630240481748162306413246647833386659341461395148758063
Підпис вірний, ключ отримано!
